# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
#Your code here
import pandas as pd
data = pd.read_csv('homepage_actions.csv')


In [5]:
#Getting an overview
print(data.head())
print(data.info())


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


In [7]:
#viewers who clicked
clicked = data[data['action'] == 'click']
viewed_and_clicked = clicked['id'].nunique()
total_viewed = data[data['action'] == 'view']['id'].nunique()
print("Number of viewers who also clicked:", viewed_and_clicked)
print("Total number of viewers:", total_viewed)


Number of viewers who also clicked: 1860
Total number of viewers: 6328


In [8]:
#anomalies ; clicked without viewing
clicked_without_viewed = clicked[~clicked['id'].isin(data[data['action'] == 'view']['id'])]
print("Clicked without viewing:")
print(clicked_without_viewed)


Clicked without viewing:
Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []


In [9]:
#overlap between control and experiment group
control_ids = data[data['group'] == 'control']['id'].unique()
experiment_ids = data[data['group'] == 'experiment']['id'].unique()
overlap = set(control_ids).intersection(experiment_ids)
print("Overlap between control and experiment groups:")
print(overlap)


Overlap between control and experiment groups:
set()


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [10]:
#Your code here
from scipy.stats import chi2_contingency
contingency_table = pd.crosstab(data['group'], data['action'])
stat, p, dof, expected = chi2_contingency(contingency_table)
print("Chi-squared test statistic:", stat)
print("P-value:", p)



Chi-squared test statistic: 3.636160051233291
P-value: 0.056537191086915774


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [11]:
#Your code here
control_group = data[data['group'] == 'control']
experiment_group = data[data['group'] == 'experiment']

control_clicks = control_group[control_group['action'] == 'click']['id'].nunique()
control_views = control_group[control_group['action'] == 'view']['id'].nunique()
control_ctr = control_clicks / control_views

experiment_views = experiment_group[experiment_group['action'] == 'view']['id'].nunique()
expected_experiment_clicks = experiment_views * control_ctr

print("Expected number of clicks for the experiment group:", expected_experiment_clicks)


Expected number of clicks for the experiment group: 838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [12]:
#Your code here
variance = experiment_views * control_ctr * (1 - control_ctr)
std_dev = variance ** 0.5
actual_experiment_clicks = experiment_group[experiment_group['action'] == 'click']['id'].nunique()
z_score = (actual_experiment_clicks - expected_experiment_clicks) / std_dev

print("Number of standard deviations from the estimate:", z_score)


Number of standard deviations from the estimate: 3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [13]:
#Your code here
from scipy.stats import norm

p_value = 1 - norm.cdf(z_score)
print("P-value based on normal distribution:", p_value)


P-value based on normal distribution: 0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

In [14]:
""" The difference in p-values could arise from the different assumptions and underlying statistical models used by the two tests. Additionally, the chi-squared test is more conservative as it considers the overall association between the variables, whereas the calculation based on the normal distribution focuses specifically on the number of clicks."""

' The difference in p-values could arise from the different assumptions and underlying statistical models used by the two tests. Additionally, the chi-squared test is more conservative as it considers the overall association between the variables, whereas the calculation based on the normal distribution focuses specifically on the number of clicks.'

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.